# CONSTRUCTION DE LA BASE DE DONNEES 

- Scraping paroles.net
- API Deezer

In [1]:
import requests
import json
import unidecode
import re
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime
import numpy as np


### Scraping paroles.net : 

In [69]:
def parse_url_lyrics(url):
    """Fonction qui permet de d'ajouter la base de l'url si manquant"""
    base = "https://www.paroles.net"
    if re.search(f"^{base}", url) is None:
        url = base + url
    return url


# def get_list_song_artist(artist):
#     """pour un artiste renvoie l'ensemble de ses plus grands succès d'après le site paroles.net
#     pour chaque chanson, on récupère le lien vers les paroles
#     """
#     artist = artist.lower()
#     base_url = f"https://www.paroles.net/{artist}"
#     get_url = requests.get(base_url)
#     if get_url.status_code == 200:
#         result_html = soup(get_url.text, 'html.parser')
#         dict_chansons = []
#         for section in ['left', 'right']:
#             try:
#                 songs = result_html.find('div', {'class': 'song-listing-extra', 'itemprop': 'tracks'}).find(
#                     'div', {'class': f'center-on-mobile best-songs box-content {section}'})
#                 list_songs = songs.findAll('a')
#                 dict_chansons = dict_chansons + [{"artist": artist, "title": song.get_text(),
#                                                   "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
#             except Exception:
#                 print(f"{artist} - Erreur dans le scraping de la liste des chansons")
#                 return None
#         return(dict_chansons)
#     else:
#         print(f"ERROR pour l'url {base_url}")
#         return None


def get_list_song_artist(artist):
    """pour un artiste renvoie l'ensemble de ses plus grands succès d'après le site paroles.net
    pour chaque chanson, on récupère le lien vers les paroles
    """
    artist = artist.lower()
    base_url = f"https://www.paroles.net/{artist}"
    get_url = requests.get(base_url)
    if get_url.status_code == 200:
        result_html = soup(get_url.text, 'html.parser')
        try:
            # songs = result_html.find('div', {'class': 'song-listing-extra', 'itemprop': 'tracks'}).find(
            # 'div', {'class': f'center-on-mobile best-songs box-content {section}'})
            # Cas 1
            songs = result_html.find(
                'div', {'class': 'song-listing-extra', 'itemprop': 'tracks'})
            list_songs = songs.findAll('a')
            dict_chansons = [{"artist": artist, "title": song.get_text(),
                              "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
            # return dict_chansons
        except Exception:
            try:
                dict_chansons = []
                # Cas 2
                # Autre liste de chansons ?
                for best_song in [' best-songs', '']:
                    for section in ['left', 'right']:
                        list_songs = [i.findAll("a") for i in result_html.findAll(
                            'div', {'class': f'center-on-mobile{best_song} box-content {section}'})][0]
                        if len(list_songs) > 1:
                            dict_chansons = dict_chansons + [{"artist": artist, "title": song.get_text(),
                                                              "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
            except Exception:
                print(f"{artist} - Erreur dans le scraping de la liste des chansons")
                return None

        for section in ['left', 'right']:
            list_songs = [i.findAll("a") for i in result_html.findAll(
                'div', {'class': f'center-on-mobile box-content {section}'})]
            if len(list_songs) == 1 :
                list_songs = list_songs[0]
                if len(list_songs) > 1:
                    dict_chansons = dict_chansons + [{"artist": artist, "title": song.get_text(),
                                                  "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
        return dict_chansons

    else:
        print(f"ERROR pour l'url {base_url}")
        return None


def get_lyrics(url_chanson):
    """récupère les paroles d'une chanson pour un url donné"""
    get_url = requests.get(url_chanson)

    if get_url.status_code == 200:
        result_html_chanson = soup(get_url.text, 'html.parser')

        paroles_list = result_html_chanson.find(
            'div', {'class': 'song-text'}).findAll('div')
        paroles_total = ""
        for paroles in paroles_list:
            paroles_total = paroles_total + paroles.text

        # paroles_total_process = paroles_total.replace(
        #     "\n", " ").replace("\r", " ").strip()
        paroles_total_process = paroles_total.replace(
            "\n", " ").replace("\r", ".").strip()
        paroles_total_process = re.sub("\s{2,}", " ", paroles_total_process)
        # TODO : ne remplacer que les espaces et non les \r
        return paroles_total_process
    else:
        print(f"ERROR pour l'url {get_url}")


def get_lyrics_artist(artist):
    """renvoie un data.frame pour un artiste donné """
    # print(artist)
    liste_chansons = get_list_song_artist(artist)
    if liste_chansons is not None:
        _ = [chanson.update({"lyrics": get_lyrics(chanson['url_lyrics'])})
             for chanson in liste_chansons]
        return pd.DataFrame.from_records(liste_chansons)
    else:
        print(f"Erreur pour l'artist {artist}")
        return None


In [61]:
def process_artist(artist):
    artist = artist.lower()
    # Enlever les lettres à accent
    artist = unidecode.unidecode(artist)
    artist = re.sub("('|&|_|\.|!)", "", artist)
    # enlever les multiples espaces
    artist = re.sub("\s{2,}", " ", artist)
    artist = artist.replace(" ", "-")
    return artist


In [62]:
df_paroles = get_list_song_artist('france-gall')


In [63]:
df_paroles


[{'artist': 'france-gall',
  'title': 'Aime-la',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-aime-la'},
 {'artist': 'france-gall',
  'title': 'Amor tambien',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-amor-tambien'},
 {'artist': 'france-gall',
  'title': 'Babacar',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-babacar'},
 {'artist': 'france-gall',
  'title': 'Bébé comme la vie',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-bebe-comme-la-vie'},
 {'artist': 'france-gall',
  'title': "C'est Bon Que Tu Sois Là",
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-c-est-bon-que-tu-sois-la'},
 {'artist': 'france-gall',
  'title': 'Calypso',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-calypso'},
 {'artist': 'france-gall',
  'title': 'Ce soir je ne dors pas',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-ce-soir-je-ne-dors-pas'},
 {'artist': 'france-gall',
  'title': 'Cézanne peint',
  'u

#### API DEEZER

In [7]:
def get_deezer_playlist(playlist_id):
    url = f"https://api.deezer.com/playlist/{playlist_id}"
    result_api = requests.get(url)
    result_api = json.loads(result_api.text)
    playlist = []
    for track in result_api['tracks']['data']:
        detail_track = {"track_id": track['id'],
                        "track_title": track['title'],
                        "artist_id": track['artist']['id'],
                        "artist": track['artist']['name'],
                        "album_id": track['album']['id'],
                        "album": track['album']['title']}
        playlist.append(detail_track)
    return pd.DataFrame(playlist)


________________________________

### Chargement des données des playlists Deezer 

In [71]:
playlist_deezer = pd.read_csv("playlists_deezer_v2.csv")
playlist_deezer.head()


,palylist_id,title
0,1999438682,ROCK FRANÇAIS - LES CLASSIQUES
1,2481548064,Les Pépites du Rock Français
2,10572497302,Rock français
3,6156189524,Rap Francais 2022 - Hit Rap Fr 2022 - Playlist...
4,6568026624,Rap Français 90/2000


In [72]:
print(playlist_deezer.shape)


(12, 2)


In [73]:
# Boucler sur une liste d'id de playlist
start = datetime.datetime.now()
list_all_playlist = [get_deezer_playlist(id) for id in playlist_deezer['palylist_id']]
df_all_playlist = pd.concat(list_all_playlist, axis=0)
end = datetime.datetime.now()
print(f"execution time : {end-start}")


execution time : 0:00:03.351112


In [74]:
print("shape : ", df_all_playlist.shape)
print("len artisit unique : ", len(df_all_playlist.artist.unique()))


shape :  (1196, 6)
len artisit unique :  404


In [76]:
df_all_playlist['artist_process'] = df_all_playlist['artist'].apply(process_artist)

artistes_to_query = df_all_playlist['artist_process'].unique()

print(len(artistes_to_query))


403


In [77]:
df_all_playlist

,track_id,track_title,artist_id,artist,album_id,album,artist_process
0,425635062,J't'emmène au vent,633,Louise Attaque,51180312,Louise Attaque (20ème anniversaire),louise-attaque
1,83923495,Ghost,372175,Skip the Use,8446997,Les Plus Grands Tubes Rock Vol 2,skip-the-use
2,77235400,L'aventurier,47,Indochine,7671262,"Tubes incontournables : Tous les tubes Rock, D...",indochine
3,73140591,BLIZZARD,56895,FAUVE,7197283,BLIZZARD,fauve
4,18049838,Jeune et con,463,Saez,1694136,Jours Etranges,saez
...,...,...,...,...,...,...,...
395,638492832,Posons-nous la question,15969,Taïro,88433282,Summer Tape,tairo
396,2150060,O C B,672,Billy Ze Kick et Les Gamins En Folie,215350,Billy Ze Kick Et Les Gamins En Folie,billy-ze-kick-et-les-gamins-en-folie
397,1244930842,Petite Marie (Fanzine Remix),1377312,Biga Ranx,207835122,Petite Marie (Fanzine Remix),biga-ranx
398,1187935552,Ça ira mieux demain,4791064,Grezou,195353002,"Les vibes de l'année, Vol. 2",grezou


In [78]:
artistes_to_query


array(['louise-attaque', 'skip-the-use', 'indochine', 'fauve', 'saez',
       'dionysos', 'calogero', 'mickey-3d', 'johnny-hallyday', 'matmatah',
       'bb-brunes', 'kyo', 'stuck-in-the-sound', 'la-femme', 'trust', 'm',
       'axel-bauer', 'les-wampas', 'superbus', 'dolly', 'patrick-coutin',
       'mademoiselle-k', 'taxi-girl', 'izia', 'deportivo',
       'jean-louis-aubert', 'fff', 'les-innocents', 'silmarils',
       'bertignac-et-les-visiteurs', 'eiffel', 'noir-desir', 'niagara',
       'astonvilla', 'no-one-is-innocent', 'la-grande-sophie', 'detroit',
       'mano-negra', 'les-garcons-bouchers', 'eddy-mitchell', 'prohom',
       'stephan-eicher', 'blankass', 'paul-personne', 'miossec',
       'louis-bertignac', 'telephone', 'tahiti-80', 'bijou',
       'no-mans-land', 'romain-humeau', 'daran-et-les-chaises',
       'les-rita-mitsouko', 'marc-seberg', 'les-fatals-picards', 'minuit',
       'experience', 'sergent-garcia', 'the-frenchies', 'asphalt-jungle',
       'stocks', 'banlie

In [79]:
start = datetime.datetime.now()
list_df_lyrics = [get_lyrics_artist(artist) for artist in artistes_to_query]
end = datetime.datetime.now()
print(f"-- execution time : {end-start}")
df_total = pd.concat(list_df_lyrics)
print(df_total.shape)
print(df_total.artist.value_counts())


saez - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist saez
silmarils - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist silmarils
bertignac-et-les-visiteurs - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist bertignac-et-les-visiteurs
les-garcons-bouchers - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist les-garcons-bouchers
stephan-eicher - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist stephan-eicher
tahiti-80 - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist tahiti-80
no-mans-land - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist no-mans-land
les-fatals-picards - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist les-fatals-picards
experience - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist experience
the-frenchies - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist the-

In [80]:
df_total


,artist,title,url_lyrics,lyrics
0,louise-attaque,Amours,https://www.paroles.net/louise-attaque/paroles...,"Amour, mon père. et je ne sais pas comment. Am..."
1,louise-attaque,Anomalie,https://www.paroles.net/louise-attaque/paroles...,"Si loin de moi, . Un grain de poussière ici ba..."
2,louise-attaque,Avec le temps,https://www.paroles.net/louise-attaque/paroles...,Aimer sur un seul pied. Sans savoir où poser. ...
3,louise-attaque,Chaque jour reste le nôtre,https://www.paroles.net/louise-attaque/paroles...,Une minute après l'autre. Chaque jour reste le...
4,louise-attaque,Depuis Toujours,https://www.paroles.net/louise-attaque/paroles...,Les jours ne sont pas éternels . Disait un ast...
...,...,...,...,...
19,dub-silence,L'hymne des légumes,https://www.paroles.net/dub-silence/paroles-l-...,"Mon style est vrai, j’ai le flow, toujours fra..."
20,dub-silence,Lovely Friends,https://www.paroles.net/dub-silence/paroles-lo...,"I am nobody, nobody, nobody,. Without my frien..."
21,dub-silence,MAJ,https://www.paroles.net/dub-silence/paroles-maj,Il y a un petit bout de temps que je vous ai r...
22,dub-silence,Matchgirl,https://www.paroles.net/dub-silence/paroles-ma...,"Demain c'est le 25 décembre, la ville s'est pa..."


In [81]:
df_paroles_artist_id = df_all_playlist[['artist', 'artist_id', 'artist_process']].drop_duplicates().merge(df_total, left_on = ['artist_process'], right_on = ['artist'])

In [82]:
df_paroles_artist_id

,artist_x,artist_id,artist_process,artist_y,title,url_lyrics,lyrics
0,Louise Attaque,633,louise-attaque,louise-attaque,Amours,https://www.paroles.net/louise-attaque/paroles...,"Amour, mon père. et je ne sais pas comment. Am..."
1,Louise Attaque,633,louise-attaque,louise-attaque,Anomalie,https://www.paroles.net/louise-attaque/paroles...,"Si loin de moi, . Un grain de poussière ici ba..."
2,Louise Attaque,633,louise-attaque,louise-attaque,Avec le temps,https://www.paroles.net/louise-attaque/paroles...,Aimer sur un seul pied. Sans savoir où poser. ...
3,Louise Attaque,633,louise-attaque,louise-attaque,Chaque jour reste le nôtre,https://www.paroles.net/louise-attaque/paroles...,Une minute après l'autre. Chaque jour reste le...
4,Louise Attaque,633,louise-attaque,louise-attaque,Depuis Toujours,https://www.paroles.net/louise-attaque/paroles...,Les jours ne sont pas éternels . Disait un ast...
...,...,...,...,...,...,...,...
13797,Dub Silence,1585174,dub-silence,dub-silence,L'hymne des légumes,https://www.paroles.net/dub-silence/paroles-l-...,"Mon style est vrai, j’ai le flow, toujours fra..."
13798,Dub Silence,1585174,dub-silence,dub-silence,Lovely Friends,https://www.paroles.net/dub-silence/paroles-lo...,"I am nobody, nobody, nobody,. Without my frien..."
13799,Dub Silence,1585174,dub-silence,dub-silence,MAJ,https://www.paroles.net/dub-silence/paroles-maj,Il y a un petit bout de temps que je vous ai r...
13800,Dub Silence,1585174,dub-silence,dub-silence,Matchgirl,https://www.paroles.net/dub-silence/paroles-ma...,"Demain c'est le 25 décembre, la ville s'est pa..."


In [85]:
df_paroles_artist_id.to_csv("paroles_artist_id_v2.csv", sep = ";", index=False)

In [83]:
df_total.artist.value_counts().head(20)  # 254/536 artistes


tairo                      213
juliette-greco             206
serge-reggiani             206
mano-solo                  206
jeanne-moreau              192
pigalle                    190
les-ogres-de-barback       188
marcel-et-son-orchestre    176
danakil                    170
alborosie                  152
no-one-is-innocent         140
les-sales-majestes         122
pierpoljak                 120
jul                        118
massilia-sound-system      110
serge-lama                 110
maxime-le-forestier        109
nolwenn-leroy              108
cali                       108
isabelle-boulay            107
Name: artist, dtype: int64

## Genre

In [86]:
def get_deezer_genre(album_id):
    url_album = f"https://api.deezer.com/album/{album_id}"
    result_album = requests.get(url_album)
    if result_album.status_code == 200:
        result_album = json.loads(result_album.text)
        if "error" in result_album:
            return {"artist_id": None,
                    "album_id": None,
                    "genres": None}
        else:
            detail_album = {"artist_id": result_album['artist']['id'],
                            "album_id": result_album['id'],
                            "genres": [g["name"] for g in result_album['genres']['data']]}
            return(detail_album)
    else:
        return None

In [89]:
def genre_majoritaire(liste_id_playlist) :
    list_all_playlist = [get_deezer_playlist(id) for id in liste_id_playlist]
    df_all_playlist = pd.concat(list_all_playlist, axis=0)
    df_all_playlist_genre = df_all_playlist['album_id'].apply(get_deezer_genre)
    df_all_playlist_genre = pd.DataFrame.from_records(list(df_all_playlist_genre))
    df_all_playlist_genre = df_all_playlist_genre[~df_all_playlist_genre['album_id'].isna()]

    df_all_playlist_genre_explode = df_all_playlist_genre.explode('genres')
    df_all_playlist_genre_explode = df_all_playlist_genre_explode.drop_duplicates()
    dict = {
        'Pop internationale' : 'Pop',
        'Pop indé/Folk' : 'Pop Indé',
        'Rock indé' : 'Rock',
        'Hard Rock' : 'Rock',
        'Rock Indé/Pop Rock' : 'Rock',
        'Dance' : 'Electro',
        'Electro Pop/Electro Rock' : 'Electro',
        'Techno/House' : 'Electro',
        'Films/Jeux vidéo' : 'Bandes originales',
        'Comédies musicales' : 'Bandes originales',
        'Musiques de films' : 'Bandes originales',
        'Rap/Hip Hop' : 'Rap',
        'Rap français' : 'Rap',
        'Metal' : 'Rock',
        'Rock français' : 'Rock',
        'Dub' : 'Reggae',
        'Ska' : 'Reggae',
        'Dancehall/Ragga' : 'Reggae',
        'Soul & Funk' : 'Soul',
        'R&B contemporain' : 'R&B'
    }
    df_all_playlist_genre_explode = df_all_playlist_genre_explode.replace({"genres" : dict})
    df_all_playlist_genre_explode = pd.concat([df_all_playlist_genre_explode, pd.get_dummies(df_all_playlist_genre_explode['genres'])], axis=1)
    df_all_playlist_genre_explode = df_all_playlist_genre_explode[~df_all_playlist_genre_explode.genres.isna()]

    unique_genres = df_all_playlist_genre_explode['genres'].unique()
    genres_by_artist = df_all_playlist_genre_explode.groupby(['artist_id'])[unique_genres].sum()
    genres_by_artist['GENRES_MAX'] = genres_by_artist.idxmax(axis=1)
    genres_by_artist['GENRES_MAX_VALUE'] = genres_by_artist[unique_genres].max(axis=1)
    genres_by_artist = genres_by_artist.reset_index()
    
    # for i in range(len(genres_by_artist['GENRES_MAX'])-1) :
    #     if genres_by_artist['GENRES_MAX'][i] =='Pop' :
    #         genres_by_artist['GENRES_MAX'][i] = genres_by_artist.loc[:, genres_by_artist.columns !='Pop'].iloc[i,:].idxmax(axis=0)
    #         unique_genres.remove('Pop')
    #         genres_by_artist['GENRES_MAX_VALUE'][i] = genres_by_artist[unique_genres].iloc[i,:].max(axis=0)

    genres_by_artist['GENRES_SUM_ROW'] = genres_by_artist[unique_genres].sum(axis=1)
    genres_by_artist['MOYENNE_ROW'] = genres_by_artist[unique_genres].replace({0: np.nan})[unique_genres].mean(axis=1)
    genres_by_artist['MAX_CERTAINTY'] = genres_by_artist.apply(lambda x :  not x['GENRES_MAX_VALUE'] != x['MOYENNE_ROW'] , axis=1)
    genres_by_artist['artist_id'] = genres_by_artist['artist_id'].astype(int)

    # return(df_all_playlist, genres_by_artist[["GENRES_MAX", "MAX_CERTAINTY"]].reset_index())
    return(genres_by_artist)




In [90]:
pd.set_option('display.max_columns', None)
liste_id_playlist = pd.read_csv("playlists_deezer_v2.csv")
liste_id_playlist = liste_id_playlist['palylist_id']
genres_by_artist = genre_majoritaire(liste_id_playlist)
genres_by_artist

,artist_id,Pop,Alternative,Rock,Chanson française,Reggae,Pop Indé,Variété Internationale,Soul,Rap,R&B,Latino,Electro,Classique,Moderne,Jazz,Bandes originales,Jeunesse,GENRES_MAX,GENRES_MAX_VALUE,GENRES_SUM_ROW,MOYENNE_ROW,MAX_CERTAINTY
0,21,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,2,2,2.00,True
1,26,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,1,1,1.00,True
2,29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,1,1,1.00,True
3,47,1,1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Rock,4,7,1.75,False
4,48,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,Rap,3,3,3.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,126265192,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rock,1,1,1.00,True
362,130356412,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,2,2,2.00,True
363,136645872,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,Rap,3,3,3.00,True
364,148696602,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,Rap,2,2,2.00,True


In [91]:
genres_by_artist.to_csv("genres_by_artist_v2.csv", sep = ";", index=False)